In [86]:
import pandas as pd
from pandarallel import pandarallel
import requests
import json
pandarallel.initialize(nb_workers=16)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read data

In [72]:
flat_prices_csv = "/home/foolingeng/ruien/data/grab-posisi/hdb-resale-prices/prices_cleaned.csv"
flat_coords_csv = "/home/foolingeng/ruien/data/grab-posisi/hdb-resale-prices/flat_coordinates_clean.csv"
flat_prices = pd.read_csv(flat_prices_csv)
flat_coords = pd.read_csv(flat_coords_csv)

### Get address

In [73]:
## Function for getting postal code, geo coordinates of addresses
def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

In [78]:
flat_prices['address'] = flat_prices['block'] + ' ' + flat_prices['street_name']
all_address = list(flat_prices['address'])
unique_address = list(set(all_address))
print('Unique addresses:', len(unique_address))

Unique addresses: 9279


In [ ]:
find_postal(unique_address, 'flat_coordinates')

### Append coordinates

In [87]:
def lookup_address(address, lookup, feat_name):
    match = (lookup['address'] == address)
    coords = lookup[feat_name][match]
    return coords.values[0]

real_value = flat_coords['address'].parallel_apply(
    lookup_address, args=(flat_prices, 'real_price',))

In [95]:
flat_coords['real_value'] = real_value
flat_coords = flat_coords.rename(columns={'LATITUDE': 'latitude', 'LONGITUDE': 'longitude'})
flat_coords.to_csv('/home/foolingeng/ruien/data/grab-posisi/hdb-resale-prices/hdb_prices_feature.csv')
flat_coords

,address,latitude,longitude,real_value
0,754 WOODLANDS CIRCLE,1.445289,103.793929,277427.490542
1,481 SEMBAWANG DR,1.452976,103.815094,296543.543626
2,506 BEDOK NTH AVE 3,1.332143,103.933720,179527.353440
3,688B CHOA CHU KANG DR,1.403436,103.750337,270684.236935
4,674 CHOA CHU KANG CRES,1.401554,103.746994,455795.889746
...,...,...,...,...
11737,862A TAMPINES ST 83,1.355064,103.936507,221904.732954
11738,173 BT BATOK WEST AVE 8,1.346105,103.740825,128049.170882
11739,327 JURONG EAST ST 31,1.349123,103.728986,261109.468913
11740,327 JURONG EAST ST 31,1.349141,103.729174,261109.468913
